In [ ]:
import sys
sys.path.insert(0, '../')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
cbc = pd.read_csv('../../../assets/silver/CBC/cbc_en_fcen_con_cluster.csv')
cbc.head(5)
cbc.shape

In [ ]:
cbc = cbc[cbc['cluster'] != 8]
cbc.shape

In [ ]:
cbc['Dni'] = cbc['Dni'].astype(str)
cbc['Dni'].nunique()

In [ ]:
cbc.dtypes

In [ ]:
datos_personales = pd.read_csv('../../../assets/silver/data_utilizada/personas_con_target.csv')
datos_personales.head(5)

In [ ]:
datos_personales['dni'].nunique()

In [ ]:
cbc

In [41]:
cbc.rename(columns={'Dni': 'dni', 'Fecha': 'fecha'}, inplace=True)

In [42]:
# Paso 0: Cargamos y aseguramos tipos
datos_personales['año_inscripcion_facultad'] = datos_personales['año_inscripcion_facultad'].astype(int)
cbc['cluster'] = cbc['cluster'].astype(int)
cbc['fecha'] = pd.to_datetime(cbc['fecha'])

# Paso 1: Verificamos clusters presentes por DNI
clusters_por_dni = cbc.groupby('dni')['cluster'].agg(lambda x: set(x)).reset_index()
clusters_por_dni['tiene_todos_los_clusters'] = clusters_por_dni['cluster'].apply(lambda x: set(range(1,7)).issubset(x))

# Paso 2: Fecha mínima por DNI (para los de 2022 con todos los clusters)
fecha_min_por_dni = cbc.groupby('dni')['fecha'].min().reset_index().rename(columns={'fecha': 'fecha_min'})

# Paso 3: Merge con datos_personales
df = datos_personales.merge(clusters_por_dni[['dni', 'tiene_todos_los_clusters']], on='dni', how='left')
df = df.merge(fecha_min_por_dni, on='dni', how='left')

# Paso 4: Asignar años estimados
def calcular_años(row):
    anio_insc = row['año_inscripcion_facultad']
    tiene_todos = row['tiene_todos_los_clusters']
    fecha_min = row['fecha_min']
    
    # Si no está en CBC (tiene_todos es NaN)
    if pd.isna(tiene_todos):
        return 2 if anio_insc == 2021 else 3
    
    if anio_insc == 2021:
        return 1 if tiene_todos else 2
    elif anio_insc == 2022:
        if tiene_todos and not pd.isna(fecha_min):
            return 2022 - fecha_min.year
        else:
            return 3
    else:
        return None  # por si hay años no contemplados

df['años_estimados'] = df.apply(calcular_años, axis=1)

In [ ]:
df

In [ ]:
df['años_estimados'].value_counts()

In [ ]:
df[df['años_estimados'] == 0]

In [ ]:
# Obtener los conteos (incluyendo NaN)
conteos = df['años_estimados'].value_counts(dropna=False).reset_index()
conteos.columns = ['años_estimados', 'count']

# Convertir NaN a string para que seaborn lo grafique
conteos['años_estimados'] = conteos['años_estimados'].astype(str)

# Crear barplot
plt.figure(figsize=(6, 4))
sns.barplot(data=conteos, x='años_estimados', y='count', palette='pastel')
plt.title('Distribución de hace cuanto hizo el CBC')
plt.xlabel('años_estimados')
plt.ylabel('Cantidad')
plt.tight_layout()
plt.show()

In [ ]:
# Obtener los conteos (incluyendo NaN)
conteos = df['años_estimados'].value_counts(dropna=False).reset_index()
conteos.columns = ['años_estimados', 'count']

# Calcular proporciones
total = conteos['count'].sum()
conteos['proporcion'] = conteos['count'] / total

# Convertir NaN a string para que seaborn lo grafique
conteos['años_estimados'] = conteos['años_estimados'].astype(str)

# Crear barplot con proporciones
plt.figure(figsize=(6, 4))
sns.barplot(data=conteos, x='años_estimados', y='proporcion', palette='pastel')
plt.title('Distribución de hace cuanto hizo el CBC (Proporción)')
plt.xlabel('años_estimados')
plt.ylabel('Proporción')
plt.tight_layout()
plt.show()

In [ ]:
# Agrupar y contar
df_plot = (
    df
    .groupby(['años_estimados', 'target'])
    .size()
    .reset_index(name='count')
)

# Crear barplot
plt.figure(figsize=(10, 6))
sns.barplot(data=df_plot, x='años_estimados', y='count', hue='target')
plt.title('Distribución de hace cuanto hizo el CBC por Cohorte')
plt.xlabel('años_estimados')
plt.ylabel('Cantidad')
plt.legend(title='Target')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Paso 1: Conteo de cada (cohorte, target)
conteos = (
    df
    .groupby(['años_estimados', 'target'])
    .size()
    .reset_index(name='count')
)

# Paso 2: Calcular proporciones por cohorte
conteos['proporcion'] = (
    conteos
    .groupby('años_estimados')['count']
    .transform(lambda x: x / x.sum())
)

# Paso 3: Plotear
plt.figure(figsize=(10, 6))
sns.barplot(data=conteos, x='años_estimados', y='proporcion', hue='target')
plt.title('Proporcion de hace cuanto hizo el CBC por Cohorte')
plt.xlabel('años_estimados')
plt.ylabel('Proporción')
plt.legend(title='Target')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
df = df[['dni', 'años_estimados']]
df.rename(columns={'años_estimados': 'tiempo_desde_cbc'}, inplace=True)
df.to_csv('../../../assets/silver/data_utilizada/tiempo_desde_cbc.csv', index=False)

In [ ]:
df['dni'].nunique()